In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy.linalg import pinv 
import networkx as nx
import pandas as pd

In [5]:
network_folder = "../datasets/STRING/networks/"
networks       = ["coocurrence", 
                  "database",
                  "fusion",
                  "neighbor"]

In [6]:
# Compute RWR matrix
def compute_rwr(A, restart_prob = 0.5):
    """
    Computing RWR matrix.
    """
    d    = A @ np.ones((A.shape[0], 1))
    
    # P  = D^-1 A
    P    = A / d
    
    n, _ = P.shape
    return pinv(np.identity(n) - restart_prob * P) * (1 - restart_prob)


In [7]:
# Combine All networks into one
df    = {}
nodes = {}
nodemaps = {}
all_nodes = set()
for net in networks:
    df[net]         = pd.read_csv(f"{network_folder}{net}.txt", sep = "\t", header = None)
    df[net]["type"] = net
    nodes[net]    = list(set(df[net][0]).union(set(df[net][1])))
    nodemaps[net] = {k: int(i) for i,k in enumerate(nodes[net])} 
    all_nodes     = all_nodes.union(set(nodes[net]))

all_df = pd.concat([df[net] for net in networks]).reset_index(drop = True)
all_df

,0,1,2,type
0,9606.ENSP00000000233,9606.ENSP00000440005,0.050,coocurrence
1,9606.ENSP00000000233,9606.ENSP00000349467,0.332,coocurrence
2,9606.ENSP00000000233,9606.ENSP00000392147,0.236,coocurrence
3,9606.ENSP00000000233,9606.ENSP00000221138,0.327,coocurrence
4,9606.ENSP00000000233,9606.ENSP00000369126,0.209,coocurrence
...,...,...,...,...
895885,9606.ENSP00000485638,9606.ENSP00000295822,0.053,neighbor
895886,9606.ENSP00000485638,9606.ENSP00000364815,0.049,neighbor
895887,9606.ENSP00000485638,9606.ENSP00000410186,0.045,neighbor
895888,9606.ENSP00000485638,9606.ENSP00000355890,0.198,neighbor


In [9]:
"""
df_in = all_df.drop([2], axis = 1).groupby([0, 1])["type"].apply(list).reset_index(name = "in")

df_in.drop(["nodes_present"], axis = 1)

def check_if_node_exists(row):
    out_row = []
    for net in networks:
        if row[0] in nodes[net] and row[1] in nodes[net]:
            out_row.append(net)
    return out_row

df_in["nodes_present"] = df_in.apply(check_if_node_exists, axis = 1)
df_in.columns = ["p", "q", "in", "nodes_present"]

df_in
"""


'\ndf_in = all_df.drop([2], axis = 1).groupby([0, 1])["type"].apply(list).reset_index(name = "in")\n\ndf_in.drop(["nodes_present"], axis = 1)\n\ndef check_if_node_exists(row):\n    out_row = []\n    for net in networks:\n        if row[0] in nodes[net] and row[1] in nodes[net]:\n            out_row.append(net)\n    return out_row\n\ndf_in["nodes_present"] = df_in.apply(check_if_node_exists, axis = 1)\ndf_in.columns = ["p", "q", "in", "nodes_present"]\n\ndf_in\n'

In [10]:
df_in = pd.read_csv("../datasets/STRING/networks/coo_fuse_data_neighbor.txt", sep = "\t")

## Computing the Adjacency matrices for each network


In [11]:
# Compute the As
As = {}
for net in networks:
    As[net] = np.zeros((len(nodemaps[net]), len(nodemaps[net])))
    for p_, q_, w, _ in df[net].values:
        p           = nodemaps[net][p_]
        q           = nodemaps[net][q_]
        As[net][p, q] = w
        As[net][q, p] = w

## Computing the RWR matrices for each network

<p> The RWR restart probability is chosen to be 0.5.</p>

In [12]:
# Compute the RWR matrices
Ps = {}
for net in networks:
    print(f"Computing RWR of the network {net}")
    Ps[net] = compute_rwr(As[net])

Computing RWR of the network coocurrence
Computing RWR of the network database
Computing RWR of the network fusion
Computing RWR of the network neighbor


<p>Saving...</p>

In [14]:
import json

OUTNET = "../datasets/STRING/RWR+ADJ"
for net in networks:
    np.save(f"{OUTNET}/{net}.adj.npy", As[net])
    np.save(f"{OUTNET}/{net}.rwr.npy", Ps[net])
    with open(f"{OUTNET}/{net}.json", "w") as oj:
        json.dump(nodemaps[net], oj)

## Adjacency matrix final computation

In [43]:
# The final output is the matrix R
all_nodeset = {k: i for i, k in enumerate(all_nodes)}

R = np.zeros((len(all_nodes), 
              len(all_nodes)))

In [64]:
import ast

for p1, q1, in_, node_pres in df_in.values:
    score = 0
    node_pres = ast.literal_eval(node_pres)    
    for net in node_pres:
        p     = nodemaps[net][p1]
        q     = nodemaps[net][q1]
        score += As[net][p, q]
        score += (Ps[net][p, q] + Ps[net][q, p]) / 2
    score /= len(node_pres)
    ap    = all_nodeset[p1]
    aq    = all_nodeset[q1]
    R[ap, aq] = score
    R[aq, ap] = score

## Saving Adjacency Matrix

In [73]:
import json

np.save("../datasets/STRING/networks/coo_fuse_data_neighbors.npy", R)
with open("../datasets/STRING/networks/coo_fuse_data_neighbors.json", "w") as jf:
    json.dump(all_nodeset, jf)

## Loading Adjacency Matrix

In [3]:
import json
import numpy as np
R = np.load("../datasets/STRING/networks/coo_fuse_data_neighbors.npy")
with open("../datasets/STRING/networks/coo_fuse_data_neighbors.json", "r") as oj:
    R_map = json.load(oj)

In [4]:
import sys
sys.path.append("../src/glide")
from glide_compute import glide_compute_map

In [5]:
"""
glide_mat, glide_map = glide_compute_map((R, 
                                          R_map))

# Saving
np.save("../datasets/STRING/networks/coo_fuse_data_neighbors.glide.npy", glide_mat)
"""

## Performing classification

In [5]:
import sys
sys.path.append("../src/scoring/")
import scoring
import predict 
import numpy as np
import json


gmat = np.load("../datasets/STRING/networks/coo_fuse_data_neighbors.glide.npy")
with open("../datasets/STRING/networks/coo_fuse_data_neighbors.json", "r") as gf:
    gmap = json.load(gf)

In [24]:
import re
import graph_io
import pandas as pd
def create_predictor_glidemat(glide_mat, k = 10, is_wt = True, confidence = True, params = {}):
    """
    GLIDE-mat to prediction
    """
    node_associations = {}
    n_nodes         = glide_mat.shape[0]
    for node in range(n_nodes):
        assoc_list = np.argsort(-glide_mat[node])[: k]
        assoc_dict = {alist: glide_mat[node, alist] for alist in assoc_list}
        node_associations[node] = assoc_dict
        # node_association : {dict protein1 -> {dict protein2 -> weight}}. number of protein1 = n_nodes,
        # number of protein2 for each protein1 = k.
    def predictor(training_labels):
        """
        Use the node_associations to find the appropriate nearest neighbors
        """
        tlabels_f = lambda i: (training_labels[i] if i in training_labels else [])
        return predict.glide(node_associations, tlabels_f, confidence = confidence)
    return predictor


def HGNC_STRING(locname):
    """
    STRING to protein name converter
    """
    df = pd.read_csv(locname, sep = "\t")
    string_name = df["#string_protein_id"]
    hgnc_name   = df["preferred_name"]
    return {key:value for key, value in zip(hgnc_name, string_name)}
    

def entrez_dict(IS_SYMBOL = True):
    if not IS_SYMBOL:
        ensp_dict = {}
        header    = True
        with open("/cluster/tufts/cowenlab/Projects/Multiple_Graphs/dataset/9606.protein.info.v11.5.txt", "r") as of:
            for line in of:
                if header:
                    header = False
                    continue
                words = re.split("\t", line.strip())
                if len(words) >= 2 and not words[1].startswith("ENSG"):
                    ensp_dict[words[1]] = words[0]
            
    s_e_dict = {}
    with open("/cluster/tufts/cowenlab/Projects/Denoising_Experiments/shared_data/dream_files/idmap.csv", "r") as of:
        header = True
        for line in of:
            if header:
                header = False
                continue
            words = re.split("\t", line.strip())
            if len(words) >= 2 and words[1] != "":
                if not IS_SYMBOL and words[0] in ensp_dict:
                    s_e_dict[ensp_dict[words[0]]] = int(words[1])
                else:
                    s_e_dict[words[0]] = int(words[1])
    rev_dict = {s_e_dict[k]: k for k in s_e_dict}
    return s_e_dict, rev_dict

def get_labels(go_type, min_level, min_prot, node_list, is_STRING = False):
    filter_protein = {"namespace": go_type, "lower_bound": min_prot}
    filter_labels  = {"namespace": go_type, "min_level": min_level}
    filter_parents = {"namespace": go_type}
    # Using entrez dict to do symbol->entrez mapping
    s_entrez, entrez_s = entrez_dict()

    if is_STRING:
        hgnc_string_map = HGNC_STRING("/cluster/tufts/cowenlab/Projects/Denoising_Experiments/9606.protein.info.v11.5.txt")
        s_entrez = {hgnc_string_map[key]: value for key, value in s_entrez.items() if key in hgnc_string_map}
        entrez_s = {value:key for key, value in s_entrez.items()}
    e_symbols          = [s_entrez[k] for k in node_list if k in s_entrez]

    f_labels, labels_dict, parent_dict = graph_io.get_go_labels_and_parents("../datasets/GO/go-basic.obo",
                                                                            "../datasets/GO/gene2go",
                                                                            filter_protein, 
                                                                            filter_labels,
                                                                            filter_parents,
                                                                            e_symbols,
                                                                            anno_map = lambda x: entrez_s[x])
    proteins_to_go     = {}
    for l in labels_dict:
        prots = labels_dict[l]
        for p in prots:
            if p not in proteins_to_go:
                proteins_to_go[p] = []
            proteins_to_go[p] += [l]
    labels = {i: proteins_to_go[node_list[i]] 
              for i in range(len(node_list)) if node_list[i] in proteins_to_go}
    return labels

In [25]:
# Computing the GO labels
labels = {}
gos = ["molecular_function", "cellular_component", "biological_process"]
for GO_TYPE in gos:
    labels[GO_TYPE] = get_labels(GO_TYPE, 5, 50, list(gmap.keys()), True)

HMS:0:00:04.489010 335,350 annotations, 20,702 genes, 18,726 GOs, 1 taxids READ: ../datasets/GO/gene2go 
18674 IDs in loaded association branch, molecular_function
  EXISTS: ../datasets/GO/go-basic.obo
../datasets/GO/go-basic.obo: fmt(1.2) rel(2021-12-15) 47,157 GO Terms; optional_attrs(relationship)
Read gene2go.dat File
Number of Labels: 40
HMS:0:00:04.650525 335,350 annotations, 20,702 genes, 18,726 GOs, 1 taxids READ: ../datasets/GO/gene2go 
18674 IDs in loaded association branch, cellular_component
  EXISTS: ../datasets/GO/go-basic.obo
../datasets/GO/go-basic.obo: fmt(1.2) rel(2021-12-15) 47,157 GO Terms; optional_attrs(relationship)
Read gene2go.dat File
Number of Labels: 85
HMS:0:00:04.655303 335,350 annotations, 20,702 genes, 18,726 GOs, 1 taxids READ: ../datasets/GO/gene2go 
18674 IDs in loaded association branch, biological_process
  EXISTS: ../datasets/GO/go-basic.obo
../datasets/GO/go-basic.obo: fmt(1.2) rel(2021-12-15) 47,157 GO Terms; optional_attrs(relationship)
Read gen

In [32]:
results = []
for go in gos:
    kfold = 5
    f1 = scoring.kfoldcv_with_pr(kfold, 
                            labels[go], 
                            create_predictor_glidemat(gmat, 
                                                    k = 10, 
                                                    is_wt = False))
    acc = scoring.kfoldcv(kfold,
                    labels[go],
                    create_predictor_glidemat(gmat, 
                                            k = 10, 
                                            is_wt = False, 
                                            confidence = False)) 
    results.append((go, f1, acc))

In [33]:
results

[('molecular_function',
  [0.5971742735941947,
   0.5968815563292742,
   0.5700622610150747,
   0.5919819866669759,
   0.5903995856925937],
  [0.5498154981549815,
   0.6005535055350554,
   0.5784132841328413,
   0.6033210332103321,
   0.5854779411764706]),
 ('cellular_component',
  [0.5482389428955244,
   0.5448515450266416,
   0.5527816406506847,
   0.5608234837731302,
   0.5535590414115613],
  [0.6093489148580968,
   0.6026711185308848,
   0.6054535336672231,
   0.5915414579855315,
   0.6238888888888889]),
 ('biological_process',
  [0.44615718123694054,
   0.42890897499174613,
   0.4337744686280131,
   0.4311428682938999,
   0.4252612557830509],
  [0.47805171377029465,
   0.48947684906794947,
   0.48947684906794947,
   0.4930847865303668,
   0.46490701859628075])]

In [30]:
acc

[0.5950184501845018,
 0.5830258302583026,
 0.5581180811808119,
 0.5913284132841329,
 0.5818014705882353]